## Libraries

In [ ]:
pip install selenium

In [ ]:
pip install requests

## Checking the site availability



**Purpose**: To verify that the site is accessible and does not return HTTP errors.

**Functionality**:
* Use Selenium to open the main page.
* Check the response status code (200 OK).
* Make sure that the title contains the correct text (for example, "ABB").

In [ ]:
# Importing the necessary libraries
import requests
from selenium import webdriver

In [ ]:
# The address of the site to check
url = "https://abb-bank.az/"

In [ ]:
# Checking if the site is accessible via HTTP
try:
    # Making a GET request
    response = requests.get(url, timeout=10)

    # Checking that the server returned 200 OK
    if response.status_code == 200:
        print("The site is available (HTTP 200 OK)")
    else:
        print(f"Error: status code received {response.status_code}")
        exit()
except Exception as e:
    print(f"Connection error: {e}")
    exit()

The site is available (HTTP 200 OK)


In [ ]:
# Selenium Settings
options = webdriver.ChromeOptions()
options.add_argument("--headless=new")
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

In [ ]:
# Checking the page content via Selenium
try:
    # Creating a browser with settings
    driver = webdriver.Chrome(options=options)

    # Opening a website
    driver.get(url)

    # Checking the page title
    page_title = driver.title
    print(f"Page title: {page_title}")

    # Checking if "ABB" is in the title
    if "ABB" in page_title:
        print("✅ The title contains 'ABB'")
    else:
        print("❌ The title does NOT contain 'ABB'")

finally:
    # Closing the browser
    try:
        driver.quit()
    except:
        pass

Page title: ABB - Müasir, Faydalı, Universal
✅ The title contains 'ABB'


## Checking the relevance of the exchange rate

**Purpose**: To automate the verification of the exchange rate table from the AMB column on the page https://abb-bank.az/az/valyuta-mezenneleri .

**Functionality**:
* Extract data from a table.
* Compare with the exchange rates of the Central Bank of Azerbaijan from the page https://www.cbar.az/currency/rates .
*Display the response when the courses differ.


**Use**: Selenium for table parsing, assert for validation

In [ ]:
import tempfile
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

In [ ]:
# Creating a temporary folder for the profile
temp_profile_dir = tempfile.mkdtemp()

In [ ]:
# Browser Settings
options = webdriver.ChromeOptions()
options.add_argument(f"--user-data-dir={temp_profile_dir}")
options.add_argument("--headless=new")
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

In [ ]:
# Creating a browser
driver = webdriver.Chrome(options=options)

In [ ]:
# === Parsing functions ===
def get_abb_rates():
    """Receives ABB Bank courses"""
    print("Loading the ABB Bank...")
    driver.get("https://abb-bank.az/az/valyuta-mezenneleri")

    try:
        # Waiting for the table
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, ".table-currency"))
        )

        table = driver.find_element(By.CSS_SELECTOR, ".table-currency")
        rows = table.find_elements(By.TAG_NAME, "tr")[2:]  # Skip the headlines

        rates = {}
        for row in rows:
            cells = row.find_elements(By.TAG_NAME, "td")
            if len(cells) < 2:
                continue

            currency = cells[0].text.strip()
            rate_text = cells[1].text.strip().replace(",", ".")

            if rate_text in ["-", ""]:
                print(f"Currency skipped: {currency} — no data available")
                continue

            try:
                # Обработка валют с масштабом (100 единиц)
                if currency.startswith("100 "):
                    base_name = currency.split(" ")[1]
                    rates[base_name] = float(rate_text) / 100
                else:
                    rates[currency] = float(rate_text)
            except ValueError:
                print(f"Incorrect course: {currency} → '{rate_text}'")

        return rates
    except Exception as e:
        print(f"Error when parsing ABB Bank: {e}")
        return {}

def get_cbar_rates():
    """Receives the rates of the Central Bank of Azerbaijan"""
    print("Loading the Central Bank of Azerbaijan...")
    driver.get("https://www.cbar.az/currency/rates")

    try:
        # Waiting for the table to appear
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, ".table_table"))
        )

        rows = driver.find_elements(By.CSS_SELECTOR, ".table_row")
        rates = {}

        for row in rows:
            try:
                valuta = row.find_element(By.CSS_SELECTOR, ".kod").text.strip()
                kurs = row.find_element(By.CSS_SELECTOR, ".kurs").text.strip().replace(",", ".")

                if kurs in ["-", ""]:
                    print(f"Currency omitted: {valuta} — no data available")
                    continue

                # Currency processing with a scale (100 units)
                if valuta.startswith("100 "):
                    base_name = valuta.split(" ")[1]
                    rates[base_name] = float(kurs) / 100
                elif valuta.startswith("10000 "):
                    base_name = valuta.split(" ")[1]
                    rates[base_name] = float(kurs) / 10000
                else:
                    kod = row.find_element(By.CSS_SELECTOR, ".kod").text.strip().upper()
                    rates[kod] = float(kurs)
            except Exception as e:
                print(f"Error in the line: {e}")
                continue

        return rates
    except Exception as e:
        print(f"Error when parsing the Central Bank: {e}")
        return {}

In [ ]:
# === Currency Matching Dictionary ===
ABB_TO_CBAR = {
    "USD": "USD",
    "EUR": "EUR",
    "GBP": "GBP",
    "RUB": "RUB",       # ABB: 100 RUB = CBAR: RUB for 100
    "CHF": "CHF",
    "JPY": "JPY",       # CBAR indicates for 100 units
    "CAD": "CAD",
    "AUD": "AUD",
    "TRY": "TRY",
    "AED": "AED",
    "SEK": "SEK",
    "NOK": "NOK",
    "DKK": "DKK",
    "KZT": "KZT",       # CBAR indicates for 100 units
    "GEL": "GEL",
    "CNY": "CNY"
}

In [ ]:
# === Main code ===
try:
    print("=== Checking the exchange rate ===")

    # Getting data from ABB Bank
    abb_rates = get_abb_rates()
    print("ABB Bank exchange rates:", abb_rates)

    # Getting data from the Central Bank of Azerbaijan
    cbar_rates = get_cbar_rates()
    print("Exchange rates of the Central Bank of Azerbaijan:", cbar_rates)

    print("\nComparing exchange rates:")

    # Checking for currency exchange rate matches
    for abb_currency, abb_rate in abb_rates.items():
        # Obtaining compliance with the Central Bank
        cbar_key = ABB_TO_CBAR.get(abb_currency)

        if not cbar_key or cbar_key not in cbar_rates:
            print(f"{abb_currency}: ⚠️ Not in the data of the Central Bank")
            continue

        cbar_rate = cbar_rates[cbar_key]

        # Scale correction for 100 units
        if abb_currency in ["RUB", "KZT", "JPY"]:
            diff = abs(abb_rate - (cbar_rate / 100))
        else:
            diff = abs(abb_rate - cbar_rate)

        if diff < 0.01:
            print(f"{abb_currency}: ✅ OK (ABB={abb_rate:.6f}, CBAR={cbar_rate:.6f})")
        else:
            print(f"{abb_currency}: ❌ DISCREPANCY! (ABB={abb_rate:.6f}, CBAR={cbar_rate:.6f}, difference={diff:.6f})")

finally:
    print("\nClosing the browser...")
    driver.quit()

=== Checking the exchange rate ===
Loading the ABB Bank...
ABB Bank exchange rates: {'USD': 1.7, 'EUR': 1.9241, 'GBP': 2.2665, 'RUB': 0.021054, 'CHF': 2.0627, 'JPY': 0.0118, 'CAD': 1.2292, 'AUD': 1.0969, 'TRY': 0.044, 'AED': 0.4628, 'SEK': 0.1765, 'NOK': 0.1645, 'DKK': 0.2579, 'KZT': 0.0033, 'GEL': 0.6198, 'CNY': 0.235}
Loading the Central Bank of Azerbaijan...
Exchange rates of the Central Bank of Azerbaijan: {'USD': 1.7, 'EUR': 1.9241, 'AUD': 1.0969, 'BYN': 0.5543, 'BGN': 0.9839, 'AED': 0.4628, 'KRW': 0.1218, 'CZK': 0.0773, 'CNY': 0.235, 'DKK': 0.2579, 'GEL': 0.6198, 'HKD': 0.2188, 'INR': 0.0201, 'GBP': 2.2665, 'IRR': 0.0306, 'SEK': 0.1765, 'CHF': 2.0627, 'ILS': 0.4747, 'CAD': 1.2292, 'KWD': 5.5433, 'KZT': 0.3304, 'QAR': 0.4663, 'KGS': 0.0195, 'HUF': 0.4757, 'MDL': 0.0995, 'NOK': 0.1645, 'UZS': 0.0132, 'PKR': 0.604, 'PLN': 0.4505, 'RON': 0.3756, 'RUB': 2.1054, 'RSD': 0.0164, 'SGD': 1.3128, 'SAR': 0.4532, 'XDR': 2.307, 'TRY': 0.044, 'TMT': 0.4857, 'UAH': 0.041, 'JPY': 1.1823, 'NZD': 1